#### 1

In [1]:
library(forecast)
library(ggplot2)
library(dplyr)
library(gridExtra)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘gridExtra’

The following object is masked from ‘package:dplyr’:

    combine



In [2]:
# library(doParallel)
library(doMC)
options(cores = 10)
registerDoMC()
require(svMisc)

Loading required package: foreach
Loading required package: iterators
Loading required package: parallel
Loading required package: svMisc

Attaching package: ‘svMisc’

The following object is masked from ‘package:utils’:

    ?



In [3]:
library(data.table)
library(hts)  


Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last



In [4]:
df_eu  <-  read.csv(file="../data_hts/hts_eu_a10.csv", header=TRUE, sep=",", row.names =1 )
df_eu  <-  df_eu/1000
var_freq  <- 4    
var_start_year  <- 2000   
gdp_agg <- df_eu_sums <-  rowSums(df_eu)
gdp_agg  <- ts(gdp_agg, frequency = var_freq, start = var_start_year) 

In [5]:
var_h  <- 8
var_window  <-  48 - 1
var_topup  <-  1
var_iter   <- (length(gdp_agg)+var_topup)/var_freq-(var_window+1)/var_freq-2

In [6]:
# var_allornot  <- var_freq
var_allornot  <- 1
# var_shift  <-  var_topup
var_shift  <-  0

var_arpdq   <-  c(2,0,0)
var_sarpdq   <-  c(1,0,0)
var_drift   <-  TRUE
var_constant   <-  TRUE

var_char  <-  c(2,1)

In [7]:
df_eu  <-  as.data.table(df_eu)
data  <-  df_eu
y  <-  hts(data, characters = var_char)
s  <-  smatrix(y)
# gts_train  <- window(y, start=20, end=67)
# gts_test  <- window(y, start=68, end=75)
# allts <- aggts(gts_train)
# allf <- matrix(, nrow=var_h, ncol=ncol(allts))

In [8]:
names_dtw  <-  readRDS('../xreg_neighbors/names_dtw_eu.rds')

In [9]:
var_countries  <-  dim(read.csv(file="../clustering/hts_eu_a10_state.csv", header=TRUE, sep=",", row.names =1 ))[2]+1
var_countries

[1] 29

In [10]:
df_regr  <- as.data.frame(y[[1]][,names_dtw[1]])
for (i in 2:length(names_dtw)){
    df_regr  <- cbind(df_regr, as.data.frame(y[[1]][,names_dtw[i]]))
}

In [11]:
colnames(df_regr)  <-  colnames(y[[1]])

In [12]:
df_regr_lag  <-  rbind(NA, df_regr[-nrow(df_regr), ])

In [13]:
dim(df_regr_lag)

[1]  75 280

In [14]:
df_regr_lag  <-  df_regr

- 2 row forecasts 
- 2 row 
- 4/12 lags 
- 4/12 diffed lags


### no weights

In [15]:
l <- ac <- m_top  <- l_test  <-  l_top  <-  list()

df_ac  <-  data.frame(matrix(ncol = 0, nrow = 0))

k  <- 0

for (i in 0:(var_iter*var_allornot - var_shift)){
    k  <-  k+1
    st_year  <-  var_start_year + i/var_allornot
    train_start  <- st_year 
    train_end  <- st_year + 1/var_freq*(var_window)
    test_start  <- st_year + 1/var_freq*(var_window+1) 
    test_end  <-  st_year + 1/var_freq*(var_window+var_h)
    
    dates  <-  (c(train_start, train_end, test_start, test_end, train_start-var_topup/var_freq, train_end-var_topup/var_freq, test_start-var_topup/var_freq)-var_start_year)*var_freq+1
    
    train_start  <- dates[1] 
    train_end  <- dates[2]
    test_start  <- dates[3]
    test_end  <- dates[4]
    
    print(c(train_start, train_end, test_start, test_end))
    
    
    if (i == 0){
        gts_train <- window(y, end=train_end)    
    }
    else{
        if ((var_shift == 0) & (i == var_iter*var_allornot - var_shift)){
            gts_train <- window(y, start=dates[5], end=dates[6])
            gts_xreg  <- df_regr_lag[dates[5]:dates[6], ]       

            
        }else{
            gts_train <- window(y, start=train_start, end=train_end)
            gts_xreg  <- df_regr_lag[train_start:train_end, ]       

    }}
    
    if ((var_shift == 0) & (i == var_iter*var_allornot - var_shift)){
        gts_test <- window(y, start=dates[7])
        gts_xreg_f  <- df_regr_lag[dates[7]:(test_end-1), ]     

      
    }else{  
     gts_test <- window(y, start=test_start, end=test_end)
        gts_xreg_f  <- df_regr_lag[test_start:test_end, ]     
          
    }
    
    allts <- aggts(gts_train)
    
    
    
    ms <-  foreach(j=(var_countries+1):ncol(allts)) %dopar% {
        model  <- tryCatch(        
        Arima(msts(allts[,j], seasonal.periods = var_freq), order=var_arpdq, seasonal=var_sarpdq,  include.drift = var_drift, include.mean = var_constant,  method="ML"), 
            error = function(e) 
                              {tryCatch( 
                Arima(msts(allts[,j], seasonal.periods = var_freq), order=var_arpdq, seasonal=var_sarpdq, include.drift = var_drift, include.mean = var_constant, method="CSS"),
                         error = function(e)              
                            auto.arima(msts(allts[,j], seasonal.periods = var_freq))
                         )}
                        
            )
        
    }
    
     
        
    allf <- matrix(, nrow=var_h, ncol=ncol(allts))
    
    
    for(j in 1:var_countries){
        allf[,j] <- rwf(msts(allts[,j], seasonal.periods = 4),h=var_h)$mean
    }

    
    for(j in (var_countries+1):ncol(allts)){  
    allf[,j] <- forecast(ms[[j-var_countries]], h=var_h)$mean
    
    }

    
    allf <- ts(allf)
    colnames(allf)  <-  colnames(allts)  
    g <- get_nodes(gts_train)
    y.f <- combinef(allf, nodes = g, weights = c(rep(0, var_countries), rep(1, (ncol(allts)-var_countries) )) )
    forecasts_7  <- y.f[[1]]
    colnames(forecasts_7)  <- colnames(y[[1]])
    data_f5  <- rbind(gts_train[[1]], forecasts_7)
    y_f5  <- hts(data_f5, characters = var_char)
        
#     if (i == 4){
#         gts_forecasts_7 <- window(y_f5, start=test_start)}
#     else{
#     gts_forecasts_7 <- window(y_f5, start=test_start, end=test_end)
# #     }

#     if ((var_shift == 0) & (i == var_iter*var_allornot - var_shift)){
#         gts_forecasts_7 <- window(y_f5, start=dates[7])
      
#     }else{  
     gts_forecasts_7 <- window(y_f5, start=dim(y_f5[[1]])[1]-var_h+1)
          
#     }
         
    print(gts_forecasts_7)
    print(gts_test)
            
    
    l[[k]]  <- gts_forecasts_7

    ac[[k]]  <-  accuracy.gts(gts_forecasts_7, gts_test)[, 1] 

    progress(i, progress.bar = TRUE)
            
    }

[1]  1 48 49 56
Hierarchical Time Series 
3 Levels 
Number of nodes at each level: 1 28 280 
Total number of series: 309 
Number of observations per series: 8 
Top level series: 
Time Series:
Start = 49 
End = 56 
Frequency = 1 
[1] 2969.945 3023.125 3023.878 3083.761 3047.698 3103.115 3106.278 3159.838
Hierarchical Time Series 
3 Levels 
Number of nodes at each level: 1 28 280 
Total number of series: 309 
Number of observations per series: 8 
Top level series: 
Time Series:
Start = 49 
End = 56 
Frequency = 1 
[1] 2958.070 3010.722 3033.490 3075.700 2977.965 3035.742 3044.227 3122.905
          0%---------25%---------50%---------75%--------100%
Progress: [1]  5 52 53 60
Hierarchical Time Series 
3 Levels 
Number of nodes at each level: 1 28 280 
Total number of series: 309 
Number of observations per series: 8 
Top level series: 
Time Series:
Start = 49 
End = 56 
Frequency = 1 
[1] 3017.996 3063.746 3072.364 3120.252 3076.105 3124.020 3133.605 3182.169
Hierarchical Time Series 
3 Le

In [16]:
xreg_forecasts  <-  l

In [17]:
xreg_forecasts[[1]][[1]]

ATA,ATB,ATF,ATG,ATJ,ATK,ATL,ATM,ATO,ATR,⋯,UKA,UKB,UKF,UKG,UKJ,UKK,UKL,UKM,UKO,UKR
1.0322651,15.60545,3.823165,16.56288,2.411012,3.352757,6.485859,6.097266,12.16554,2.007248,⋯,2.720559,62.24125,26.78799,76.31916,28.18527,37.46291,56.28894,49.42649,85.62853,17.11843
1.1657585,15.78788,4.473388,16.56654,2.376745,3.375019,6.545082,6.544128,12.20452,2.035804,⋯,2.766165,60.92328,26.95979,75.77347,27.89478,38.06426,55.89173,49.42938,83.87006,16.90934
1.0495325,15.85518,4.664319,16.71796,2.396542,3.383787,6.606707,6.583171,12.22909,2.055490,⋯,2.703683,60.17768,27.38700,76.19965,28.07348,38.44551,55.86864,49.82960,84.81168,17.08347
0.9771642,16.01909,4.950414,16.85669,2.494133,3.410739,6.663838,6.875887,12.33530,2.061383,⋯,2.709124,60.33609,27.79309,76.64192,28.44481,39.53273,56.09278,50.40215,86.43465,17.27377
1.0385771,16.07256,3.936002,16.98028,2.486987,3.418565,6.721510,6.403440,12.42368,2.080459,⋯,2.716815,59.87824,28.04874,77.10329,28.53577,40.62335,56.01389,50.77277,87.53645,17.48763
1.1361515,16.20670,4.553244,17.06767,2.461833,3.433471,6.779407,6.820481,12.47755,2.102813,⋯,2.690163,59.18580,28.06680,76.93337,28.45861,41.10434,55.74070,51.01932,87.54755,17.49102
1.0553250,16.29363,4.734005,17.20124,2.478220,3.446210,6.837559,6.860148,12.52067,2.120994,⋯,2.645851,58.65197,28.19318,77.07334,28.51162,41.48780,55.55539,51.31169,88.28331,17.60218
1.0052386,16.41816,5.006365,17.33112,2.553398,3.464598,6.895177,7.135723,12.62168,2.132884,⋯,2.625572,58.33342,28.32441,77.21477,28.62171,42.13595,55.43365,51.61919,89.18266,17.71360


### no weights, neighbor reg lag(1)

In [18]:
l <- ac <- m_top  <- l_test  <-  l_top  <-  list()

df_ac  <-  data.frame(matrix(ncol = 0, nrow = 0))

k  <- 0

for (i in 0:(var_iter*var_allornot - var_shift)){
    k  <-  k+1
    st_year  <-  var_start_year + i/var_allornot
    train_start  <- st_year 
    train_end  <- st_year + 1/var_freq*(var_window)
    test_start  <- st_year + 1/var_freq*(var_window+1) 
    test_end  <-  st_year + 1/var_freq*(var_window+var_h)
    
    dates  <-  (c(train_start, train_end, test_start, test_end, train_start-var_topup/var_freq, train_end-var_topup/var_freq, test_start-var_topup/var_freq)-var_start_year)*var_freq+1
    
    train_start  <- dates[1] 
    train_end  <- dates[2]
    test_start  <- dates[3]
    test_end  <- dates[4]
    
    print(c(train_start, train_end, test_start, test_end))
    
    
    if (i == 0){
        gts_train <- window(y, end=train_end)    
    }
    else{
        if ((var_shift == 0) & (i == var_iter*var_allornot - var_shift)){
            gts_train <- window(y, start=dates[5], end=dates[6])
            gts_xreg  <- df_regr_lag[dates[5]:dates[6], ]       

            
        }else{
            gts_train <- window(y, start=train_start, end=train_end)
            gts_xreg  <- df_regr_lag[train_start:train_end, ]       

    }}
    
    if ((var_shift == 0) & (i == var_iter*var_allornot - var_shift)){
        gts_test <- window(y, start=dates[7])
        gts_xreg_f  <- df_regr_lag[dates[7]:(test_end-1), ]     

      
    }else{  
     gts_test <- window(y, start=test_start, end=test_end)
        gts_xreg_f  <- df_regr_lag[test_start:test_end, ]     
          
    }
    
    allts <- aggts(gts_train)
    
    
    
    ms <-  foreach(j=(var_countries+1):ncol(allts)) %dopar% {
        model  <- tryCatch(        
        Arima(msts(allts[,j], seasonal.periods = var_freq), order=var_arpdq, seasonal=var_sarpdq,  include.drift = var_drift, include.mean = var_constant, xreg = gts_train[[1]][,names_dtw[j-var_countries]],  method="ML"), 
            error = function(e) 
                              {tryCatch( 
                Arima(msts(allts[,j], seasonal.periods = var_freq), order=var_arpdq, seasonal=var_sarpdq, include.drift = var_drift, include.mean = var_constant, xreg = gts_train[[1]][,names_dtw[j-var_countries]] , method="CSS"),
                         error = function(e)              
                            auto.arima(msts(allts[,j], seasonal.periods = var_freq))
                         )}
                        
            )
        
    }
    
     
        
    allf <- matrix(, nrow=var_h, ncol=ncol(allts))
    
    
    for(j in 1:var_countries){
        allf[,j] <- rwf(msts(allts[,j], seasonal.periods = 4),h=var_h)$mean
    }

    
    
    
    for(j in (var_countries+1):ncol(allts)){  
    allf[,j] <- tryCatch(forecast(ms[[j-var_countries]], h=var_h, xreg=xreg_forecasts[[k]][[1]][,names_dtw[j-var_countries]])$mean,
            error = function(e)
                forecast(ms[[j-var_countries]], h=var_h)$mean
                            
                        )
    }

    
    allf <- ts(allf)
    colnames(allf)  <-  colnames(allts)  
    g <- get_nodes(gts_train)
    y.f <- combinef(allf, nodes = g, weights = c(rep(0, var_countries), rep(1, (ncol(allts)-var_countries) )) )
    forecasts_7  <- y.f[[1]]
    colnames(forecasts_7)  <- colnames(y[[1]])
    data_f5  <- rbind(gts_train[[1]], forecasts_7)
    y_f5  <- hts(data_f5, characters = var_char)
        
#     if (i == 4){
#         gts_forecasts_7 <- window(y_f5, start=test_start)}
#     else{
#     gts_forecasts_7 <- window(y_f5, start=test_start, end=test_end)
# #     }

#     if ((var_shift == 0) & (i == var_iter*var_allornot - var_shift)){
#         gts_forecasts_7 <- window(y_f5, start=dates[7])
      
#     }else{  
     gts_forecasts_7 <- window(y_f5, start=dim(y_f5[[1]])[1]-var_h+1)
          
#     }
         
    print(gts_forecasts_7)
    print(gts_test)
            
    
    l[[k]]  <- gts_forecasts_7

    ac[[k]]  <-  accuracy.gts(gts_forecasts_7, gts_test)[, 1] 

    progress(i, progress.bar = TRUE)
            
    }

[1]  1 48 49 56


Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xr

“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Pl

“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Pl

“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Pl

...

“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Pl

“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Pl

“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Pl

“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Please check that the regressors are in the same order.”Warning message in forecast.Arima(ms[[j - var_countries]], h = var_h, xreg = xreg_forecasts[[k]][[1]][, :
“xreg contains different column names from the xreg used in training. Pl

Hierarchical Time Series 
3 Levels 
Number of nodes at each level: 1 28 280 
Total number of series: 309 
Number of observations per series: 8 
Top level series: 
Time Series:
Start = 49 
End = 56 
Frequency = 1 
[1] 3367.746 3298.531 3355.648 3352.524 3415.808 3369.042 3427.297 3433.883
Hierarchical Time Series 
3 Levels 
Number of nodes at each level: 1 28 280 
Total number of series: 309 
Number of observations per series: 8 
Top level series: 
Time Series:
Start = 68 
End = 75 
Frequency = 1 
[1] 3395.389 3357.919 3432.684 3434.182 3530.894 3471.013 3542.181 3541.582


In [19]:
var_acc_metrics  <-  c(1,2,4)
df_acc_test  <- ac[[1]][var_acc_metrics]
for (i in ac[2:length(ac)]){
    x  <-  i[var_acc_metrics]
    df_acc_test   <-  cbind(df_acc_test, x)
}
df_acc_test  <- t(df_acc_test)
df_acc_test  <-  as.data.frame(df_acc_test)
row.names(df_acc_test)  <-  seq(1,length(ac))
df_acc  <- df_acc_test
df_acc  <-  cbind(df_acc, seq(1, length(ac)))
colnames(df_acc)[length(var_acc_metrics)+1]  <-  'Iteration'

In [20]:
df_acc$Iteration  <-  df_acc$Iteration*var_freq-(var_freq-1)

In [21]:
df_acc$Iteration[length(df_acc$Iteration)]  <-   df_acc$Iteration[length(df_acc$Iteration)] - var_topup

In [22]:
df_acc

ME,RMSE,MAPE,Iteration
-34.083446,43.69866,1.1537843,1
-9.979855,34.14499,0.9165031,5
87.403287,103.88215,2.6522099,9
74.680377,78.99228,2.2364626,13
-54.699663,61.23114,1.6264233,17
85.670672,90.37010,2.4607772,20


In [23]:
colMeans(df_acc)

ME      RMSE      MAPE Iteration 
24.831895 68.719887  1.841027 10.833333

In [28]:
df_mean_acc <- readRDS("df_mean_acc.rds")
rownames(df_mean_acc)[1:5]  <-  c('1 level', 'OLS weights', 'NO weights', 'NO weights, neighbor reg' ,'NO weights, forecast reg' )
df_mean_acc

,Квартальные,Сезонно сглаженные,Месячные
1 level,64.30372,299.5702,15884.63
OLS weights,64.24503,298.8763,15346.86
NO weights,64.23140,298.8923,15342.24
"NO weights, neighbor reg",64.10458,299.4158,15511.31
"NO weights, forecast reg",64.25570,297.5359,15338.88


In [31]:
df_mean_acc[3,1]  <- readRDS("df_mean_acc_no_weights.rds")[2,1]


In [32]:
df_mean_acc[4,1]  <-  colMeans(df_acc)[[2]] 
df_mean_acc

,Квартальные,Сезонно сглаженные,Месячные
1 level,64.30372,299.5702,15884.63
OLS weights,64.24503,298.8763,15346.86
NO weights,66.31314,298.8923,15342.24
"NO weights, neighbor reg",68.71989,299.4158,15511.31
"NO weights, forecast reg",64.25570,297.5359,15338.88
